In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

%matplotlib inline 

/home/sunild/.local/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [3]:
data=pd.read_csv('ime692_project.csv')
# avoid State and Count name
cols=data.columns[2:].to_list()
data=data[cols]
# seperate data into test and train based on `Test` indicator
pd.options.mode.chained_assignment = None  # default='warn' --> to supress chain assignment warning <--
test_data=data.loc[data.Test==1] ## previous indices could be useful ##
train_data=data.loc[data.Test==0]
print("==========================================")
print("No of testing observations: ",test_data.shape[0])
print("No of training observations: ",train_data.shape[0])
print("==========================================")

# drop the indicator inplace
test_data.drop('Test',axis=1,inplace=True)
train_data.drop('Test',axis=1,inplace=True)

# sepearate predicted and regressor variables
y_test_data=test_data['CvdVax_DisparityY'].tolist()
X_test_data=test_data.drop('CvdVax_DisparityY',axis=1)
y_train_data=train_data['CvdVax_DisparityY'].tolist()
X_train_data=train_data.drop('CvdVax_DisparityY',axis=1)


No of testing observations:  225
No of training observations:  531


In [4]:
from sklearn.preprocessing import StandardScaler
scalerInst=StandardScaler()
X_train_data_normalized=scalerInst.fit_transform(X_train_data)
X_test_data_normalized=scalerInst.fit_transform(X_test_data)

In [5]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train_data_normalized,y_train_data)
print('R2 >>>',lm.score(X_test_data_normalized,y_test_data))

R2 >>> 0.3605625338226661


In [6]:
MSE=np.mean((lm.predict(X_test_data_normalized)-y_test_data)**2)
print('MSE >>>',MSE)

MSE >>> 68.35394429129394


In [13]:
from sklearn.linear_model import RidgeCV,Ridge
r=np.linspace(-3,10,100)
alphas=np.power(10*np.ones(100),r)
rcv=RidgeCV(alphas=alphas,cv=5)
rcv.fit(X_train_data_normalized,y_train_data)
cvAlpha=rcv.alpha_
print('alpha >>>',cvAlpha)
rlm=Ridge(alpha=cvAlpha)
rlm.fit(X_train_data_normalized,y_train_data)
print('R2 >>>',rlm.score(X_test_data_normalized,y_test_data))
MSE=np.mean((rlm.predict(X_test_data_normalized)-y_test_data)**2)
print('MSE >>>',MSE)

alpha >>> 39.442060594376564
R2 >>> 0.35040969731597005
MSE >>> 69.43925201516895


In [14]:
from sklearn.linear_model import Lasso,LassoCV
r=np.linspace(-3,10,100)
alphas=np.power(10*np.ones(100),r)
rcv=LassoCV(alphas=alphas,cv=5)
rcv.fit(X_train_data_normalized,y_train_data)
cvAlpha=rcv.alpha_
print('alpha >>>',cvAlpha)
rlm=Lasso(alpha=cvAlpha)
rlm.fit(X_train_data_normalized,y_train_data)
print('R2 >>>',rlm.score(X_test_data_normalized,y_test_data))
MSE=np.mean((rlm.predict(X_test_data_normalized)-y_test_data)**2)
print('MSE >>>',MSE)

alpha >>> 0.037649358067924694
R2 >>> 0.3616769518949525
MSE >>> 68.23481634703123


In [22]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(random_state=1) # to reproduce results
rfr.fit(X_train_data_normalized,y_train_data)
print('R2 >>>',rfr.score(X_test_data_normalized,y_test_data))
MSE=np.mean((rfr.predict(X_test_data_normalized)-y_test_data)**2)
print('MSE >>>',MSE)

R2 >>> 0.4250175946019502
MSE >>> 61.46389191426086


- no improvment even with ridge regression with cv